### Simple RAG app with web parser

In [2]:
import os 
# get env variables 
from dotenv import load_dotenv
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('GOOGLE_API_KEY') 
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
# langsmith project tracker


Roadmap :

data ingestion - web parser

data split into chunks - llm token limitation 

creating embeddings 

vector store 

retrival

In [4]:
# web parser in langchain 
from langchain_community.document_loaders import WebBaseLoader # internally uses bs4

In [1]:
# budget webpage from MINT website
webpage = "https://www.livemint.com/budget/budget-2024-25-live-updates-nirmala-sitharaman-speech-income-tax-infra-agriculture-pli-tax-exemption-india-economy-gdp-11721667773635.html"

# try it out with different webpages

In [7]:
laoder = WebBaseLoader(webpage)
docs = laoder.load()
print(docs)

[Document(metadata={'source': 'https://www.livemint.com/budget/budget-2024-25-live-updates-nirmala-sitharaman-speech-income-tax-infra-agriculture-pli-tax-exemption-india-economy-gdp-11721667773635.html', 'title': 'Budget 2024 Live Updates: ‘Want to simplify approach to taxation’, says Nirmala Sitharaman on Lower capital gains | Mint', 'description': "Budget 2024 Live Updates: FM Nirmala Sitharaman presented the Modi 3.0 govt's Budget today, introducing increased standard deduction and revised tax rates for salaried individuals under the new tax regime.", 'language': 'en'}, page_content='\n\n\n\n          \n\n\n\n\n\n\nBudget 2024 Live Updates: ‘Want to simplify approach to taxation’, says Nirmala Sitharaman on Lower capital gains | Mint\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n  \n\n\n\n\n\n  \n\n\n\n\n \n\n\n\n\n\n\n\n          Explore          Sign in      e-paper Subscribe  Sign In     Thursday, 25 July 2024              Stocks       Mutual Funds       News    

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter 

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)

document_op = text_splitter.split_documents(docs)


In [20]:
document_op[10].page_content

'has paved a clear path towards a Viksit Bharat – bringing together the power of the private and public sector, and reinforcing this with our distinctive demographic advantage."      23 Jul 2024, 09:31:32 PM IST Budget 2024 Live: Srini Chinamilli Co-founder and CEO Tessolve, said, Budget 2024 Live: "This year\'s budget is forward-looking and aimed at making India self-reliant while equipping our youth with the skills necessary for employment. The government has announced various plans to tailor courses in collaboration with industry, fostering strong industry-academia partnerships to enhance skill development. Its plan for freshers and internships will also help the new and potential employees. Tessolve is proud to have contributed to upskilling individuals over the past few years. With the government support, this effort will further boost emerging sectors like semiconductors, addressing the skill gap effectively.Additionally, investments in research, including funding for the'

[help]: ggogleai embeddings : 'https://python.langchain.com/v0.2/docs/integrations/text_embedding/google_generative_ai/#usage'

In [22]:
# generating embeddings

from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [23]:
# vector store using FAISS , we can also use Chroma or any other cloud based vector store

from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(document_op, embeddings)

In [24]:
vectorstore

In [29]:
query="What was the budget allocated for bihar in 2024 budget"
result=vectorstore.similarity_search(query)


In [36]:
print(result[1].page_content)

to 15%."   23 Jul 2024, 12:09:46 PM IST Budget 2024 Live: Space economy also in focus Budget 2024 Live: Fin Min also added that there will be continued focus on the expansion of space economy by five times over the span of 10 years. She further went on saying that a venture capital fund of ₹1,000 crore will be established to support the goal.   23 Jul 2024, 12:06:05 PM IST Budget 2024 Live: To provide financial support for flood management  Budget 2024 Live: The Finance Minister acknowledged that Bihar has often been affected by floods and noted the lack of progress on plans to build flood control structures in Nepal. The government will allocate financial support estimated at ₹11,500 crore. Assam, which faces annual floods, will receive aid for flood management and related projects. Himachal Pradesh, having experienced extensive flood damage, will get support for reconstruction through multilateral assistance. Additionally, Uttarakhand, which has suffered significant damage from


In [2]:
# we can add a document chain ,  retriver chain to the vector store to get more relevant results.
# we can make a simple stream lit app to demonstrate the results.